## 0. Setup

### 0.1 Install PySpark

In [ ]:
! sudo apt-get update
! sudo mkdir -p /usr/share/man/man1
! sudo apt-get install -y openjdk-11-jdk
! pip install pyspark==3.2.1

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession
            .builder
            .config(
                "spark.jars",
                "jars/singlestore-jdbc-client-1.0.1.jar, \
                jars/singlestore-spark-connector_2.12-4.0.0-spark-3.2.0.jar, \
                jars/spray-json_3-1.3.6.jar"
            )
            .getOrCreate()
        )

In [ ]:
spark.version

### 0.2 Connect to SingleStore DB

In [ ]:
server = "<TO DO>"
password = "<TO DO>"
 
port = "3306"
cluster = server + ":" + port

In [ ]:
spark.conf.set("spark.datasource.singlestore.ddlEndpoint", cluster)
spark.conf.set("spark.datasource.singlestore.user", "admin")
spark.conf.set("spark.datasource.singlestore.password", password)
spark.conf.set("spark.datasource.singlestore.disablePushdown", "false")

## 1. Apache Spark

### 1.1 Create Spark Dataframe

In [ ]:
iris_df = spark.read.csv(
                    "data/iris.csv",
                    header = True,
                    inferSchema = True
                )

In [ ]:
iris_df.show(5)

### 1.2 Exploratory Data Analysis (EDA)

In [ ]:
(iris_df
    .groupBy("species")
    .count()
    .show()
)

In [ ]:
(iris_df
    .describe(
        "sepal_length",
        "sepal_width",
        "petal_length",
        "petal_width"
    )
    .show()
)

In [ ]:
# https://plotly.com/python/pca-visualization/

import plotly.express as px
from sklearn.decomposition import PCA
import pandas as pd

pandas_iris_df = iris_df.toPandas()

X = pandas_iris_df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]

pca = PCA(n_components = 2)
components = pca.fit_transform(X)

pca_fig = px.scatter(
    components,
    x = 0,
    y = 1,
    color = pandas_iris_df["species"]
)

pca_fig.show()

In [ ]:
# More about correlations:
# https://www.w3schools.com/python/pandas/pandas_correlations.asp

import matplotlib.pyplot as plt
import seaborn as sns

sns.heatmap(
    pandas_iris_df.corr(),
    cmap = "OrRd",
    annot = True
)

plt.title("Correlations")
plt.plot()

### 1.3 Write Spark Dataframe to SingleStore DB

In [ ]:
(iris_df.write
    .format("singlestore")
    .option("loadDataCompression", "LZ4")
    .mode("overwrite")
    .save("iris_db.iris")
)

## 2. MindsDB

### 2.1 Create Spark Dataframe

In [ ]:
from pyspark.sql.types import *

btc_schema = StructType([
    StructField("ds", DateType(), True),
    StructField("y", FloatType(), True)
])

In [ ]:
btc_df = spark.read.csv(
                    "data/btc_data.csv",
                    header = True,
                    dateFormat = "YYYY-MM-DD",
                    schema = btc_schema
                )

In [ ]:
btc_df.show(5)

### 2.2 Write Spark Dataframe to SingleStore DB

In [ ]:
(btc_df.write
    .format("singlestore")
    .option("loadDataCompression", "LZ4")
    .mode("overwrite")
    .save("btc_db.btc")
)

### 2.3 Create MindsDB Datasource

In [ ]:


df_1 = _deepnote_execute_sql("""DROP DATASOURCE btc_data;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_1

In [ ]:


df_2 = _deepnote_execute_sql("""CREATE DATASOURCE btc_data
WITH ENGINE = "singlestore",
    PARAMETERS = {
        "user" : "admin",
        "password" : "<TO DO>",
        "host" : "<TO DO>",
        "port" : 3306,
        "database" : "btc_db"
    }
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_2

In [ ]:


df_3 = _deepnote_execute_sql("""SELECT *
FROM btc_data.btc
ORDER BY ds
LIMIT 5;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_3

### 2.4 Create MindsDB Predictor

In [ ]:


df_4 = _deepnote_execute_sql("""DROP PREDICTOR mindsdb.btc_pred;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_4

In [ ]:


df_5 = _deepnote_execute_sql("""CREATE PREDICTOR btc_pred
FROM btc_data
    (SELECT * FROM btc)
PREDICT y
ORDER BY ds
WINDOW 1;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_5

In [ ]:


df_6 = _deepnote_execute_sql("""SHOW PREDICTORS;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_6

In [ ]:


df_7 = _deepnote_execute_sql("""DESCRIBE btc_pred;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_7

In [ ]:


df_8 = _deepnote_execute_sql("""DESCRIBE btc_pred.features;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_8

### 2.5 Show MindsDB Predictions

In [ ]:


pred_10_df = _deepnote_execute_sql("""SELECT orig_table.ds AS date, pred_table.y AS predicted_y, orig_table.y AS actual_y
FROM btc_data.btc AS orig_table
JOIN mindsdb.btc_pred AS pred_table
WHERE orig_table.ds > '2011-12-30'
LIMIT 10;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
pred_10_df

In [ ]:
line_fig = px.line(
                pred_10_df,
                x = "date",
                y = ["actual_y", "predicted_y"]
            )

line_fig.show()

In [ ]:


df_10 = _deepnote_execute_sql("""SELECT orig_table.ds AS date, pred_table.y AS predicted_y, orig_table.y AS actual_y
FROM btc_data.btc AS orig_table
JOIN mindsdb.btc_pred AS pred_table
WHERE orig_table.ds > LATEST;
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
df_10

In [ ]:


pred_all_df = _deepnote_execute_sql("""SELECT orig_table.ds AS date, pred_table.y AS predicted_y, orig_table.y AS actual_y
FROM btc_data.btc AS orig_table
JOIN mindsdb.btc_pred AS pred_table
WHERE orig_table.ds > '2011-12-30';
""", 'SQL_4A3995E6_E6C8_4CF8_AA3B_E89C56745F8F')
pred_all_df

In [ ]:
btc_pred_df = spark.createDataFrame(pred_all_df)

In [ ]:
(btc_pred_df.write
    .format("singlestore")
    .option("loadDataCompression", "LZ4")
    .mode("overwrite")
    .save("btc_db.pred")
)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=342fcb27-ae26-4515-a337-32741b0b3da9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>